## Corner 1: Terrestrial LiDAR Filtering (using Python to process .ply files)
Name: Divya Natekar

Net ID: dyn2009

NYU ID: N19974330

This Python program simulates a terrestrial LiDAR scanner placed near Corner 1 of the Jay–Willoughby point cloud. It performs ray-based filtering by dividing the 3D space into small angular sectors. For each angular direction (ray), the code retains only the furthest visible point, simulating what a ground-based scanner would see (removing occlusions like people, poles, or cars that block the view).
Retained points are visualized in grayscale and removed points in green, with the scanner origin marked by a blue sphere.

In [3]:
import open3d as o3d
import numpy as np

# ---------- 1. Load Point Cloud ----------
file_path = "jay-willoughby.ply"
pcd = o3d.io.read_point_cloud(file_path)

if not pcd.has_points():
    raise ValueError(f"Could not load points. File not found or empty: {file_path}")

points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# ---------- 2. Simulate Scanner at Corner 1 (near bottom-left) ----------
origin = np.array([
    points[:, 0].min() + 1.0,
    points[:, 1].min() + 1.0,
    points[:, 2].min() + 0.5
])

# ---------- 3. Calculate Vectors from Scanner ----------
vectors = points - origin
distances = np.linalg.norm(vectors, axis=1)
unit_vectors = vectors / distances[:, np.newaxis]

# ---------- 4. Bin by Azimuth and Elevation ----------
azimuth = np.degrees(np.arctan2(unit_vectors[:, 1], unit_vectors[:, 0]))
elevation = np.degrees(np.arcsin(unit_vectors[:, 2]))

az_bins = np.round(azimuth).astype(int)
el_bins = np.round(elevation).astype(int)

# ---------- 5. Keep Only the Farthest Point Per Ray ----------
ray_dict = {}
for i, (az, el, d) in enumerate(zip(az_bins, el_bins, distances)):
    key = (az, el)
    if key not in ray_dict or d > ray_dict[key][1]:
        ray_dict[key] = (i, d)

kept_indices = [idx for idx, _ in ray_dict.values()]
removed_indices = list(set(range(len(points))) - set(kept_indices))

# ---------- 6. Retained = Green ----------
retained_pcd = pcd.select_by_index(kept_indices)
green = np.tile([0.0, 1.0, 0.0], (len(kept_indices), 1))
retained_pcd.colors = o3d.utility.Vector3dVector(green)

# ---------- 7. Removed = Greyscale ----------
removed_pcd = pcd.select_by_index(removed_indices)
intensity = np.mean(colors[removed_indices], axis=1)
grey = np.stack([intensity]*3, axis=1)
removed_pcd.colors = o3d.utility.Vector3dVector(grey)

# ---------- 8. Scanner Visualization ----------
scanner = o3d.geometry.TriangleMesh.create_sphere(radius=0.5)
scanner.translate(origin)
scanner.paint_uniform_color([0, 0, 1])  # blue

# ---------- 9. Visualize ----------
vis = o3d.visualization.Visualizer()
vis.create_window("Corner 1 - Terrestrial LiDAR", width=1600, height=1000)
opt = vis.get_render_option()
opt.background_color = np.array([1.0, 1.0, 1.0])  # white background
opt.point_size = 1.0

vis.add_geometry(retained_pcd)
vis.add_geometry(removed_pcd)
vis.add_geometry(scanner)

vis.run()
vis.destroy_window()
